In [1]:
#Build a movie recomendation engine using python

In [3]:
#import the libraries 
import re
import pandas as pd 
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

In [4]:
#Load the data
df = pd.read_csv("title.basics.tsv", sep='\t')
print(df.shape)
df.head(3)

(130064, 9)


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0030298,tvMovie,Julius Caesar,Julius Caesar,0,1938,\N,101,"Drama,History"
1,tt0038309,tvMovie,As You Like It,As You Like It,0,1946,\N,\N,Drama
2,tt0038738,tvMovie,A Midsummer Night's Dream,A Midsummer Night's Dream,0,1946,\N,150,Drama


In [5]:
#Split the data into train and test 
train, test = train_test_split(df, test_size=0.1,shuffle=False)
test['Movie_id'] = range(0,test.shape[0])
test=test.set_index('Movie_id')
test.shape

<ipython-input-5-6c76016dbdd7>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['Movie_id'] = range(0,test.shape[0])


(13007, 9)

In [6]:
#Select the most important colums in your Data
columns = ['genres','originalTitle','isAdult','primaryTitle']

In [7]:
test[columns].tail(10)

,genres,originalTitle,isAdult,primaryTitle
Movie_id,,,,
12997,Comedy,Fuckboi Romeo,0,Fuckboi Romeo
12998,"Comedy,Drama",Try Again,0,Try Again
12999,"Documentary,History",Fascism i en svart bil,0,Fascism i en svart bil
13000,Comedy,Hot Explicit Ladies,0,Hot Explicit Ladies
13001,Music,"Adidas x Pusha T at Art Basel, Miami",0,"Adidas x Pusha T at Art Basel, Miami"
13002,Biography,Danielle Darrieux: Il est poli d'être gai!,0,Danielle Darrieux: Il est poli d'être gai!
13003,"Action,Horror,Thriller",Mahiwaga,0,Mahiwaga
13004,Comedy,Pink Taxi,0,Pink Taxi
13005,Family,March of Dimes Presents: Once Upon a Dime,0,March of Dimes Presents: Once Upon a Dime


In [8]:
#Check for any missing values in the important columns
if test[columns].isnull().values.any():
    test=test.dropna()
test[columns].isnull().values.any()

False

In [10]:
#Combine the important colums into a single string
def get_important_features(data):
    important_features = []
    for index,row in data.iterrows():
        new_column = re.sub('[^A-Za-z0-9]+', ' ',row['genres'])\
        +' '+re.sub('[^A-Za-z0-9]+', ' ',row['originalTitle'])\
        +' '+re.sub('[^A-Za-z0-9]+', ' ',row['primaryTitle'])\
        +' '+str(row['isAdult'])
        important_features.append(new_column.lower())
    return important_features

In [12]:
#Create a column to hold the combined strings
test["important_features"] = get_important_features(test)

#show the resulted data
test.head(10)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,important_features
Movie_id,,,,,,,,,,
0,tt6908630,tvMovie,Broadcast Battleground,Broadcast Battleground,0,2014,\N,\N,Comedy,comedy broadcast battleground broadcast battle...
1,tt6908858,tvMovie,Der Islamreport,Der Islamreport,0,2016,\N,45,Documentary,documentary der islamreport der islamreport 0
2,tt6909136,tvMovie,EquinoXe,EquinoXe,1,2017,\N,75,Adult,adult equinoxe equinoxe 1
3,tt6909272,tvMovie,The Sheriff and the Astronaut,The Sheriff and the Astronaut,0,1984,\N,\N,Action,action the sheriff and the astronaut the sheri...
4,tt6909598,tvMovie,Moonys Kindchen weint nicht,Moonys Kindchen weint nicht,0,1961,\N,30,\N,n moonys kindchen weint nicht moonys kindchen...
5,tt6909776,tvMovie,...wie am Schnürchen,...wie am Schnürchen,0,1961,\N,\N,\N,n wie am schn rchen wie am schn rchen 0
6,tt6909902,tvMovie,Dash for Cash,Dash for Cash,0,\N,\N,\N,Action,action dash for cash dash for cash 0
7,tt6910058,tvMovie,Yuki-no-bo Mountain Zaô,'Yuki no bô' no yama Zaô,0,1992,\N,26,Documentary,documentary yuki no b no yama za yuki no bo ...
8,tt6910162,tvMovie,Tartuffe,Tartuffe,0,1964,\N,\N,Drama,drama tartuffe tartuffe 0


In [13]:
#Convert the text to a matrix of token counts 
cm = CountVectorizer().fit_transform(test['important_features'])

In [14]:
#Get the cosine similraity matrix from the count matrix
cs = cosine_similarity(cm)
print(cs)

[[1.         0.         0.         ... 0.11111111 0.         0.        ]
 [0.         1.         0.         ... 0.         0.         0.        ]
 [0.         0.         1.         ... 0.         0.         0.        ]
 ...
 [0.11111111 0.         0.         ... 1.         0.         0.        ]
 [0.         0.         0.         ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]]


In [15]:
#Get the shape of the cosine similar
cs.shape

(13007, 13007)

In [26]:
#Get the title of the movie to compare with
test_title ="Art of Falling in Love"

#Find the movie id 
movie_id = test[test.originalTitle==test_title].index.values[0]
print(movie_id)


12870


In [27]:
#Create a list of enumeration of the movies similarity scores [(movie_id, similarity_score)]
scores = list(enumerate(cs[movie_id]))
print(len(scores[1:]))
print(scores[0:10])

13006
[(0, 0.0), (1, 0.0), (2, 0.0), (3, 0.0), (4, 0.0), (5, 0.0), (6, 0.0), (7, 0.0), (8, 0.0), (9, 0.0)]


In [28]:
#Sort the list
sorted_scores = sorted(scores, key = lambda x:x[1], reverse= True)
sorted_scores = sorted_scores[1:]
print(len(sorted_scores))
print(scores[0:10])

13006
[(0, 0.0), (1, 0.0), (2, 0.0), (3, 0.0), (4, 0.0)]


In [29]:
#Create a loop to print the top 10 similar movies 
j=0
print('Top ten recomendations to movie (', test_title,') are:\n')
for item in sorted_scores:
    movie_title = test[test.index ==item[0]]['originalTitle'].values[0]
    print(j,movie_title)
    j=j+1
    if j>10:
        break
    

Top ten recomendations to movie ( Art of Falling in Love ) are:

0 In Love
1 Rome in Love
2 Frozen in Love
3 Love, of Course
4 Christmas in Love
5 Nature of Love
6 Love in Design
7 The Art of Us
8 Love Is in the Air
9 The Summer of Love
10 The Edge of Art


In [31]:
def sort_scores(movie_id):
    scores = list(enumerate(cs[movie_id]))
    #Sort the list
    sorted_scores = sorted(scores, key = lambda x:x[1], reverse= True)
    return sorted_scores[1:]

def print_top_ten(sorted_scores, movie_title):
    #Create a loop to print the top 10 similar movies 
    j=0
    print('Top ten recomendations to movie (', movie_title,') are:\n')
    for item in sorted_scores:
        movie_title = test[test.index ==item[0]]['originalTitle'].values[0]
        print(j,movie_title)
        j=j+1
        if j>10:
            break
    
def see_top_ten_movies_for(some_movie):
    print('----------------------------------')
    print('Looking for the movie: '+ some_movie)
    movie_id = None
    try:
    #Find the movie id 
        movie_id = test[test.originalTitle==some_movie].index.values[0]
    except:
        print('Movie not Found')
    if movie_id is not None:
        print('Found it!')
        scores = sort_scores(movie_id)
        print_top_ten(scores, some_movie)
    print('----------------------------------')
        


In [32]:
see_top_ten_movies_for('Star Wars')

----------------------------------
Looking for the movie: Star Wars
Movie not Found
----------------------------------


In [33]:
see_top_ten_movies_for('Alien Galaxy')

----------------------------------
Looking for the movie: Alien Galaxy
Found it!
Top ten recomendations to movie ( Alien Galaxy ) are:

0 Alien Agenda
1 American Alien
2 The Galaxy Britain Built
3 Confessions of an Alien Abductee
4 1-O
5 Galaxy Express 999
6 The Galaxy Fighter Bushiban
7 Closing In: The Hunt for Alien Life
8 Alien Life and Where to Find It
9 Galaxy Ball Team Earth
10 My Mom is an Alien
----------------------------------


In [34]:
see_top_ten_movies_for('Fast Cars')

----------------------------------
Looking for the movie: Fast Cars
Found it!
Top ten recomendations to movie ( Fast Cars ) are:

0 Hold Fast
1 Nowhere Fast
2 Battle Cars Live
3 Fast ein Poet
4 The Fast Lane
5 Goodwood: Classic Cars
6 Fast perfekt verliebt
7 Cars: In Stereo
8 N.O.C.
9 2 x 2 = 4 - Eine fast versehentliche Sendung
10 Ron James: Fast Forward
----------------------------------


In [40]:
# import csv
# with open('title.basics.tsv', 'r') as inp, open('title.basics_result.tsv', 'w') as out:
#     writer = csv.writer(out,delimiter="\t")
#     firstline=True
#     for row in csv.reader(inp,delimiter="\t"):
#         if firstline:    #skip first line
#             firstline = False
#             writer.writerow(row)
#             continue
#         else:
#             if "Episode" not in row[2] and 'Movie' in row[1]:
#                 writer.writerow(row)

test[test['primaryTitle'].str.contains('Love')].head(10)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,important_features
Movie_id,,,,,,,,,,
149,tt6937070,tvMovie,Love at the Shore,Love at the Shore,0,2017,\N,84,"Comedy,Family,Romance",comedy family romance love at the shore love a...
209,tt6948128,tvMovie,All for Love,All for Love,0,2017,\N,83,"Comedy,Romance",comedy romance all for love all for love 0
411,tt6968356,tvMovie,Love Is in the Air,Love Is in the Air,0,2017,\N,93,Romance,romance love is in the air love is in the air 0
514,tt6978344,tvMovie,Kipper: Puppy Love,Kipper: Puppy Love,0,1997,\N,46,"Animation,Comedy",animation comedy kipper puppy love kipper pupp...
816,tt7034316,tvMovie,"Eat, Play, Love","Eat, Play, Love",0,2017,\N,83,"Comedy,Romance",comedy romance eat play love eat play love 0
824,tt7035242,tvMovie,In Love,In Love,0,2016,\N,\N,Romance,romance in love in love 0
884,tt7049178,tvMovie,Rome in Love,Rome in Love,0,2019,\N,90,Romance,romance rome in love rome in love 0
1005,tt7078236,tvMovie,Pee Wee Lovell,Pee Wee Lovell,0,2016,\N,75,Documentary,documentary pee wee lovell pee wee lovell 0
1220,tt7129736,tvMovie,The Love Echo,The Love Echo,0,2011,\N,\N,Comedy,comedy the love echo the love echo 0


In [37]:
 test[test['primaryTitle'].str.contains('Alien')].head(40)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,important_features
Movie_id,,,,,,,,,,
370,tt6964168,tvMovie,World War A: Aliens Invade Earth,World War A: Aliens Invade Earth,0,2016,\N,55,Documentary,documentary world war a aliens invade earth wo...
2109,tt7319174,tvMovie,Closing In: The Hunt for Alien Life,Closing In: The Hunt for Alien Life,0,2017,\N,58,Documentary,documentary closing in the hunt for alien life...
3139,tt7555314,tvMovie,My Mom is an Alien,My Mom is an Alien,0,2008,\N,\N,Animation,animation my mom is an alien my mom is an alien 0
3703,tt7669926,tvMovie,American Alien,American Alien,0,\N,\N,\N,Comedy,comedy american alien american alien 0
6428,tt8262520,tvMovie,Illegal Aliens,Illegal Aliens,0,2018,\N,\N,Drama,drama illegal aliens illegal aliens 0
6763,tt8356848,tvMovie,Alien Life and Where to Find It,Alien Life and Where to Find It,0,2018,\N,60,Documentary,documentary alien life and where to find it al...
6934,tt8408382,tvMovie,Confessions of an Alien Abductee,Confessions of an Alien Abductee,0,2013,\N,50,Documentary,documentary confessions of an alien abductee c...
9439,tt9032972,tvMovie,Alien Agenda,Alien Agenda,0,\N,\N,61,\N,n alien agenda alien agenda 0
9687,tt9066452,tvMovie,Alien Galaxy,Alien Galaxy,0,2018,\N,\N,Documentary,documentary alien galaxy alien galaxy 0
